In [1]:
########################################################################################################################
# This script visualizes the decision curve analysis for point prediction models
########################################################################################################################

In [ ]:
########################################################################################################################
# Import packages
########################################################################################################################
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import warnings
mpl.rcParams["font.family"] = "sans-serif"
mpl.rcParams["font.sans-serif"] = ["Century Gothic"]
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
########################################################################################################################
# USER_SPECIFIC SETTING
# ABLATION: Boolean. False for pre-ablation modeling and True for post-ablation modeling
# IMPUTE: A string representing the imputation method adopted (in 'Zero', 'Mean', and 'Median')
# CSL: Boolean. False for standard learning and True for cost-sensitive learning
# IN_DIR: The input path of the directories storing the modeling results 
# (e.g., the directory with the subdirectory ANN_Results/) 
# OUT_DIR: The output path of the directory storing the overall performance statistics
# ALGO_LIST: List of the point-prediction algorithms 
########################################################################################################################
ABLATION: bool = False
IMPUTE: str = 'Zero'
CSL: bool = False
IN_DIR: str = ''
OUT_DIR: str = '_Final_Results/'
ALGO_LIST: list[str] = ['ANN', 'EN', 'LogReg', 'SVM', 'XGB']

In [ ]:
########################################################################################################################
# Specify the directories to extract probability estimates
########################################################################################################################
ablate_str: str = '_Ablated' if ABLATION else ''
csl_str: str = '_CSL' if CSL else ''
files_needed: dict[str, str] = {}
for algo in ALGO_LIST:
    dir_path: str = os.path.join(IN_DIR, f'{algo}_Results{ablate_str}/Predicted_Probabilities/')
    file_path: str = dir_path + f'1_encounters_60_days_{IMPUTE}{ablate_str}{csl_str}.csv'
    files_needed[algo] = file_path    

In [3]:
########################################################################################################################
# Define a function to compute standardized net benefit (sNB)
########################################################################################################################
def nb(y_true_, y_prob_, pt, standard=False):
    y_pred_ = (y_prob_ >= pt).astype(int)
    TP = np.sum((y_pred_ == 1) & (y_true_ == 1))
    FP = np.sum((y_pred_ == 1) & (y_true_ == 0))
    n = len(y_true_)
    nb = (TP / n) - (FP / n) * (pt / (1 - pt))
    snb = nb / np.mean(y_true_)
    return snb if standard else nb

In [4]:
########################################################################################################################
# Define other relevant functions to compute sNB (from counts and treat all)
########################################################################################################################
def nb_from_counts(tp, fp, n, pt, prevalence, standard=True):
    nb = (tp / n) - (fp / n) * (pt / (1 - pt))
    return nb / prevalence if standard else nb

def compute_treat_all_nb(y_true, pts, standard=True):
    n = len(y_true)
    prev = np.mean(y_true)
    tp = np.sum(y_true == 1)
    fp = np.sum(y_true == 0)
    return [nb_from_counts(tp, fp, n, pt, prev, standard=standard) for pt in pts]


In [8]:
########################################################################################################################
# Create dictionaries to save true labels, probabilities, and sNB values
########################################################################################################################
true_dict: dict = {}
prob_dict: dict = {}
nb_dict: dict = {}

In [7]:
########################################################################################################################
# Define the policy thresholds for the computation of sNB
########################################################################################################################
POLICY_PROBS = np.array([i / 1000 for i in range(1, 1000)], dtype=float)
assert np.all((POLICY_PROBS > 0) & (POLICY_PROBS < 1)), 'Each element in POLICY_PROBS must be strictly within (0, 1)'

In [55]:
########################################################################################################################
# Load the probability estimates files
########################################################################################################################
for algo, file_path in files_needed.items():
    assert os.path.exists(file_path), file_path
    df_cur: pd.DataFrame = pd.read_csv(file_path)

    ####################################################################################################################
    # Extract the true labels and estimated probabilities
    # Compute and store the standardized net benefit (sNB) for each decision threshold in POLICY_PROBS
    ####################################################################################################################
    y_test = df_cur['y_test'].to_numpy()
    y_prob = df_cur['y_prob'].to_numpy()

    true_dict[algo] = y_test
    prob_dict[algo] = y_prob
    nb_dict[algo] = [nb(y_test, y_prob, pt, standard=True) for pt in POLICY_PROBS]

In [ ]:
########################################################################################################################
# Create the plot
########################################################################################################################
algo_ref = next(iter(true_dict.keys()))
y_true_ref = true_dict[algo_ref]
prev = np.mean(y_true_ref)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 5))
for algo, nb_vals in nb_dict.items():
    algo_name = algo.split('_CSL')[0]
    algo_name_cur = algo_name if algo_name != 'LogReg' else 'LR'
    ax.plot(POLICY_PROBS, np.asarray(nb_vals, dtype=float), label=algo_name_cur, linewidth=2, alpha=0.8)

treat_all_vals = np.asarray(compute_treat_all_nb(y_true_ref, POLICY_PROBS, standard=True), dtype=float)
ax.plot(POLICY_PROBS, treat_all_vals, linestyle='--', linewidth=3.5, label='Treat all', zorder=2, alpha=0.9)
ax.axhline(0.0, linestyle='-', linewidth=6, label='Treat none', zorder=1, color='black', alpha=0.9)
ax.set_xlabel('Threshold probability (p' + r'$_t$' +')', fontsize=15)
y_label = 'Standardized net benefit (sNB)'
ax.set_ylabel(y_label, fontsize=15, labelpad=10)

ax.set_ylim(-0.05, 1.05)
ax.set_xlim(0, 0.6 if CSL else 0.5)      # Revise if needed
ax.grid(alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=13)
handles, labels = ax.get_legend_handles_labels()
seen = set()
handles_u, labels_u = [], []
for h, lab in zip(handles, labels):
    if lab not in seen:
        handles_u.append(h)
        labels_u.append(lab)
        seen.add(lab)
    
fig.legend(handles_u, labels_u, loc='upper center', bbox_to_anchor=(0.5, 0.98), ncols=7, frameon=True, fontsize=14, 
           title_fontproperties={'size': 14, 'weight': 'bold'})
plt.tight_layout(rect=[0, 0, 1, 0.9])
out_path = os.path.join(OUT_DIR, f"DCA_Point_sNB_{IMPUTE}{ablate_str}{csl_str}.png")
plt.savefig(out_path, dpi=300, bbox_inches="tight")
plt.show()